# Análisis completo combinando filtrado, agrupación y merge 

## importar

In [1]:
# importar librerías
import pandas as pd
import numpy as np

## Creación de Dataset

In [17]:
# Dataset de ventas
ventas = pd.DataFrame({
    'id_venta': range(1, 11),
    'id_cliente': np.random.choice([1, 2, 3, 4, 5], 10),
    'id_producto': np.random.choice([101, 102, 103, 104], 10),
    'cantidad': np.random.randint(1, 5, 10),
    'fecha': pd.date_range('2024-01-01', periods=10, freq='D')
})

# Dataset de clientes
clientes = pd.DataFrame({
    'id_cliente': [1, 2, 3, 4, 5],
    'nombre': ['Ana', 'Carlos', 'María', 'Juan', 'Luis'],
    'ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Sevilla']
})

# Dataset de productos
productos = pd.DataFrame({
    'id_producto': [101, 102, 103, 104],
    'nombre': ['Laptop', 'Mouse', 'Teclado', 'Monitor'],
    'precio': [1200, 25, 80, 300],
    'categoria': ['Electrónica', 'Accesorios', 'Accesorios', 'Electrónica']
})

print("Datasets creados:")
print(f"Ventas: {ventas.shape}")
print(f"Clientes: {clientes.shape}")
print(f"Productos: {productos.shape}")

Datasets creados:
Ventas: (10, 5)
Clientes: (5, 3)
Productos: (4, 4)


## Filtrado avanzado con query():

In [8]:
# Ventas del mes actual con query
ventas_recientes = ventas.query('fecha >= "2024-01-05"')
print(f"\nVentas recientes: {len(ventas_recientes)}")




Ventas recientes: 6


In [9]:
# Productos caros usando variable externa
precio_limite = 100
productos_caros = productos.query('precio >= @precio_limite')
print(f"Productos caros (>= {precio_limite}): {productos_caros['nombre'].tolist()}")

Productos caros (>= 100): ['Laptop', 'Monitor']


## Agrupación y agregación:

In [10]:
# Ventas por producto
ventas_por_producto = ventas.groupby('id_producto')['cantidad'].sum()
print(f"\nVentas por producto:\n{ventas_por_producto}")




Ventas por producto:
id_producto
101    4
102    4
103    8
104    6
Name: cantidad, dtype: int32


In [11]:
# Estadísticas por cliente
stats_por_cliente = ventas.groupby('id_cliente').agg({
    'cantidad': ['sum', 'mean'],
    'id_venta': 'count'
})
print(f"\nEstadísticas por cliente:\n{stats_por_cliente}")


Estadísticas por cliente:
           cantidad      id_venta
                sum mean    count
id_cliente                       
1                10  2.5        4
2                 1  1.0        1
3                 3  3.0        1
4                 6  3.0        2
5                 2  1.0        2


## Merge para análisis completo:

In [13]:
# Unir ventas con productos
ventas_productos = pd.merge(ventas, productos, on='id_producto')

# Calcular totales
ventas_productos['total'] = ventas_productos['cantidad'] * ventas_productos['precio']

# Unir con clientes
analisis_completo = pd.merge(ventas_productos, clientes, on='id_cliente')

print(f"\nAnálisis completo (primeras 5 filas):\n{analisis_completo.head()}")




Análisis completo (primeras 5 filas):
   id_venta  id_cliente  id_producto  cantidad      fecha nombre_x  precio  \
0         1           1          103         3 2024-01-01  Teclado      80   
1         2           3          102         3 2024-01-02    Mouse      25   
2         3           4          104         3 2024-01-03  Monitor     300   
3         4           1          103         2 2024-01-04  Teclado      80   
4         5           4          101         3 2024-01-05   Laptop    1200   

     categoria  total nombre_y    ciudad  
0   Accesorios    240      Ana    Madrid  
1   Accesorios     75    María    Madrid  
2  Electrónica    900     Juan  Valencia  
3   Accesorios    160      Ana    Madrid  
4  Electrónica   3600     Juan  Valencia  


In [12]:
# Análisis por ciudad
ventas_por_ciudad = analisis_completo.groupby('ciudad')['total'].sum()
print(f"\nVentas totales por ciudad:\n{ventas_por_ciudad}")


Ventas totales por ciudad:
ciudad
Barcelona    1200
Madrid       1315
Sevilla       325
Valencia     4500
Name: total, dtype: int64


## Filtrado final:

In [16]:
# Clientes con compras > 1000
clientes_top = analisis_completo.groupby(['id_cliente', 'nombre'])['total'].sum()
clientes_top = clientes_top[clientes_top > 1000]
print(f"\nClientes con compras > 1000:\n{clientes_top}")

KeyError: 'nombre'